In [108]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold

In [109]:
train_data = pd.read_csv('Train.csv')
test_data = pd.read_csv('Test.csv')
train_data.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


## Exploratory Data Analysis

In [110]:
print('Train data shape', train_data.shape)
print('Test data shape', test_data.shape)

Train data shape (56000, 52)
Test data shape (24000, 51)


In [111]:
train_data.default_status.value_counts()

no     42285
yes    13715
Name: default_status, dtype: int64

## Fill the missing values

In [112]:
train_data = train_data.fillna(-999)
test_data = test_data.fillna(-999)

### Convert the categorical features to binary

In [113]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
train_data["form_field47"] = enc.fit_transform(train_data["form_field47"])
train_data["default_status"] = enc.fit_transform(train_data["default_status"])

test_data["form_field47"] = enc.fit_transform(test_data["form_field47"])

In [114]:
#picking X and y
X = train_data.drop(["Applicant_ID", "default_status"], axis =1)
y = train_data["default_status"]

X_test = test_data.drop(["Applicant_ID"], axis = 1)

## Split data into train and validation sets

In [115]:
from sklearn.model_selection import train_test_split  #change test size from .20 to .28

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, stratify = y, random_state = 42)

X_train.shape, X_val.shape

((44800, 50), (11200, 50))

In [116]:
filter = VarianceThreshold(0.5)
X_train = filter.fit_transform(X_train)
X_val = filter.transform(X_val)
X_test = filter.transform(X_test)

X_train.shape, X_val.shape, X_test.shape

((44800, 49), (11200, 49), (24000, 49))

In [117]:
scaler = StandardScaler()   # MinMaxScaler
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

## Model Building

In [118]:
import random

In [119]:
seed = 1994
random.seed(seed)
np.random.seed(seed)

In [120]:
def crossVal(model):
    avg = 0
    for fold, (tr, te) in enumerate(skfolds.split(xtrainval, ytrainval), 1):
        train_x, test_x = xtrainval.iloc[tr], xtrainval.iloc[te]
        train_y, test_y = ytrainval.iloc[tr], ytrainval.iloc[te]
        
        model.fit(train_x, train_y, eval_set=[(train_x, train_y), (test_x, test_y)], 
                  early_stopping_rounds=200, verbose=False)
        preds = model.predict_proba(test_x)[:,1]
        logloss = log_loss(test_y, preds)
        print(f'Fold {fold}: Logloss: {logloss}')
        avg += logloss
    free = gc.collect()
    print(f'Avg Logloss: {avg/fold}')

In [121]:
def crossValPred(model):
    avg = 0
    test_scores = []
    for fold, (tr, te) in enumerate(skfolds.split(X, y), 1):
        train_x, test_x = X.iloc[tr], X.iloc[te]
        train_y, test_y = y.iloc[tr], y.iloc[te]
        
        model.fit(train_x, train_y, eval_set=[(train_x, train_y), (test_x, test_y)], 
                  early_stopping_rounds=200, verbose=False)
        
        preds = model.predict_proba(test_x)[:, 1]
        logloss = log_loss(test_y, preds)
        print(f'Fold {fold}: Logloss: {logloss}')
        avg += logloss
        
        
        test_pred = model.predict_proba(test_c)[:,1]
        test_scores.append(test_pred)
    print(f'Avg Logloss: {avg/fold}')
    free = gc.collect()
    return np.mean(test_scores, 0)

In [122]:
def plot_graph(model, xtr, ytr, ax=None):
    fig, ax = plt.subplots(1,2,figsize=(10,4))
    plot_confusion_matrix(model, xtr, ytr, ax=ax[0])
    ax[0].grid(False)
    plot_roc_curve(model, xtr, ytr, ax=ax[1])

In [123]:
xtrainval, xtest, ytrainval, ytest = train_test_split(X, y, test_size=0.18, stratify=y, random_state=seed)
xtrain, xval, ytrain, yval = train_test_split(xtrainval, ytrainval, test_size=0.20, stratify=ytrainval, random_state=seed)

In [124]:
xtrain.shape, xval.shape, xtest.shape

((36736, 50), (9184, 50), (10080, 50))

In [125]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, plot_confusion_matrix, plot_roc_curve
print("done")

done


In [126]:
import catboost as cat

In [127]:
cat_cols = train_data.select_dtypes('object').columns.drop(['Applicant_ID'], 1)


In [128]:

catm = cat.CatBoostClassifier(random_seed=seed, cat_features=cat_cols, bootstrap_type='Bernoulli', 
                              iterations=1000, silent=True, eval_metric='Logloss')

In [129]:
skfolds= StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)

In [130]:

xtrainval, xtest, ytrainval, ytest = train_test_split(X, y, test_size=0.18, stratify=y, random_state=seed)
xtrain, xval, ytrain, yval = train_test_split(xtrainval, ytrainval, test_size=0.20, stratify=ytrainval, random_state=seed)

In [131]:

catm = cat.CatBoostClassifier(random_seed=seed, cat_features=cat_cols, bootstrap_type='Bernoulli', 
                              iterations=1000, silent=True, eval_metric='Logloss')

In [132]:
%%time
catm.fit(xtrain, ytrain, eval_set=[(xval,yval)], early_stopping_rounds=350, verbose=200); #stopping from 350 to 300

Learning rate set to 0.077177
0:	learn: 0.6450292	test: 0.6453607	best: 0.6453607 (0)	total: 58.2ms	remaining: 58.2s
200:	learn: 0.3811010	test: 0.4085066	best: 0.4084831 (199)	total: 9.26s	remaining: 36.8s
400:	learn: 0.3525937	test: 0.4090435	best: 0.4079676 (249)	total: 18.4s	remaining: 27.5s
Stopped by overfitting detector  (350 iterations wait)

bestTest = 0.4079676266
bestIteration = 249

Shrink model to first 250 iterations.
Wall time: 29.4 s


In [133]:
import os, gc, warnings
warnings.filterwarnings('ignore')


In [134]:
crossVal(catm)

Fold 1: Logloss: 0.415200598550914
Fold 2: Logloss: 0.4076531913946591
Fold 3: Logloss: 0.40836322115753537
Fold 4: Logloss: 0.40089085659113066
Fold 5: Logloss: 0.40233139441861854
Avg Logloss: 0.4068878524225715


In [135]:
gc.collect()

22

In [136]:
test_c = test_data.drop(['Applicant_ID'], 1)

In [137]:
cat_pred = crossValPred(catm)


Fold 1: Logloss: 0.40795356317427
Fold 2: Logloss: 0.4045227484708526
Fold 3: Logloss: 0.40332120094547697
Fold 4: Logloss: 0.40553864612585794
Fold 5: Logloss: 0.40825247246659907
Avg Logloss: 0.40591772623661126


# LGB


In [138]:
xtrainval, xtest, ytrainval, ytest = train_test_split(X, y, test_size=0.18, stratify=y, random_state=seed)
xtrain, xval, ytrain, yval = train_test_split(xtrainval, ytrainval, test_size=0.28, stratify=ytrainval, random_state=seed)


In [139]:
lgbm = lgb.LGBMClassifier(random_state=seed, n_estimators=1000, metric='binary_logloss', learning_rate=0.04, subsample=0.7,
                         colsample_bytree=0.8, num_leaves=64, reg_alpha=4, min_child_samples=5,subsample_freq=2)

In [140]:
lgbm.fit(xtrain, ytrain, eval_set=[(xtrain, ytrain), (xval,yval)], verbose=200, early_stopping_rounds=200);

Training until validation scores don't improve for 200 rounds
[200]	training's binary_logloss: 0.326336	valid_1's binary_logloss: 0.412365
Early stopping, best iteration is:
[167]	training's binary_logloss: 0.337911	valid_1's binary_logloss: 0.412005


In [141]:
crossVal(lgbm)

Fold 1: Logloss: 0.4164871267322893
Fold 2: Logloss: 0.40713020202436345
Fold 3: Logloss: 0.41067884124274906
Fold 4: Logloss: 0.4012310255321854
Fold 5: Logloss: 0.4051385074063975
Avg Logloss: 0.408133140587597


In [142]:

gc.collect()

22

In [143]:
lgb_pred = crossValPred(lgbm)

Fold 1: Logloss: 0.4079291342537728
Fold 2: Logloss: 0.4059193993137731
Fold 3: Logloss: 0.40352824087943223
Fold 4: Logloss: 0.4062568066075599
Fold 5: Logloss: 0.40962283433407737
Avg Logloss: 0.40665128307772314


In [144]:
blend = (0.6 * lgb_pred + 0.4)

In [145]:

test_id = test_data.Applicant_ID

In [146]:
lgb_df =  pd.DataFrame({'Applicant_ID' : test_id, 'default_status' : lgb_pred})
cat_df =  pd.DataFrame({'Applicant_ID' : test_id, 'default_status' : cat_pred})
blend_df =  pd.DataFrame({'Applicant_ID' : test_id, 'default_status' : blend})

In [147]:
lgb_df.to_csv('lgblast.csv', index=False)
cat_df.to_csv('catlast.csv', index=False)
blend_df.to_csv('blendla.csv', index=False)

In [149]:

gc.collect()

22

In [153]:
pip install notebook-as-pdf

Note: you may need to restart the kernel to use updated packages.
